# Using the VGG16 Convolutional Base for CIFAR-10 Dataset

### The CIFAR-10 Dataset  
* The CIFAR-10 dataset (Canadian Institute For Advanced Research) is a collection of images that are commonly used to train machine learning and computer vision algorithms  
* It is one of the most widely used datasets for machine learning research  
* The CIFAR-10 dataset contains 60,000 32x32 color images in 10 different classes  
* The 10 different classes represent airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks  
* There are 6,000 images of each class  

In [0]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [0]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [0]:
import matplotlib.pyplot as plt
plt.imshow(x_train[4])

In [0]:
from keras.utils import to_categorical
train_labels = to_categorical( y_train )
test_labels = to_categorical( y_test )
print(train_labels.shape)
print(test_labels.shape)

## Method1: Develop our own ConvNet



In [0]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers

model = models.Sequential()
model.add( layers.Conv2D( 16, ( 3, 3 ), activation = 'sigmoid', input_shape = x_train[0, :, :, :].shape ) )
model.add( layers.Conv2D( 4, ( 3, 3 ), activation = 'sigmoid' ) )
model.add( layers.Flatten() )
model.add( layers.Dense( 10, activation = 'softmax' ) )
model.summary()

In [0]:
model.compile( optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = [ 'accuracy' ] )
history = model.fit( x_train, train_labels, epochs = 64, batch_size = 128, validation_data = (x_test, test_labels))

In [0]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## Method2: Use the VGG16 'Convolutional Base'

In [0]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

In [0]:
conv_base.summary()

In [0]:
x_test_vgg_output = conv_base.predict(x_test, verbose = 1)
x_train_vgg_output = conv_base.predict(x_train, verbose = 1)

In [0]:
print(x_test_vgg_output.shape)
print(x_train_vgg_output.shape)

In [0]:
import numpy as np
x_test_new = np.reshape(x_test_vgg_output, (10000, 1 * 1 * 512))
x_train_new = np.reshape(x_train_vgg_output, (50000, 1 * 1 * 512))

In [0]:
print(x_test_new.shape)
print(x_train_new.shape)

In [0]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers

new_model = models.Sequential()
new_model.add(layers.Dense(256, activation='relu', input_dim = 1 * 1 * 512))
new_model.add(layers.Dropout(0.5))
new_model.add(layers.Dense(10, activation='softmax'))

new_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = new_model.fit(x_train_new, train_labels, epochs=64, batch_size=128, validation_data = (x_test_new, test_labels))

In [0]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()